In [21]:
import io
import json

import geopandas
import pandas as pd

from statistics import mean

from ipywidgets import HTML

from ipyleaflet import Map, MarkerCluster, Marker, Popup

# Load or request geojson
# json = requests.get("...").json()
with open("examples/sample.geojson") as fh:
    json = json.load(fh)

# GeoDataFrame from JSON
gdf = geopandas.GeoDataFrame.from_features(json)

#gdf.dtypes
#geometry    geometry
#date          object
#username      object
#geosong       object
#dtype: object

# Beware of the line below:
# "TypeError: Object of type Timestamp is not JSON serializable"
# gdf["date"] = pd.to_datetime(gdf["date"])

# Center map on the mean latitude/longitude
center = (mean(gdf.geometry.y),
          mean(gdf.geometry.x))
m = Map(center=center, zoom=5)

# Create markers from GeoDataFrame
# NOTE: we cannot use GeoData layer because popups do not seem to be supported
#       see https://github.com/jupyter-widgets/ipyleaflet/issues/837
markers = []
for index, row in gdf.iterrows():
    marker = Marker(location=(row.geometry.y, 
                              row.geometry.x))
    marker.popup = HTML(value="<b>{}</b><br/>{}<br/>".format(row.geosong["title"], \
                                                             row.geosong["artist"]))
    markers.append(marker)

m.add_layer(MarkerCluster(markers=markers))

display(m)

Map(center=[41.02621495450895, -0.39550781250000006], controls=(ZoomControl(options=['position', 'zoom_in_text…